In this notebook, we will be walking through feature selection by using Lasso Regression. 

[Dataset](https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/data)

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler

In [2]:
# Load dataset
train = pd.read_csv('/Users/tsawaengsri/Desktop/Data Science Courses/Datasets/paribas/train.csv')
test = pd.read_csv('/Users/tsawaengsri/Desktop/Data Science Courses/Datasets/paribas/test.csv')
train.shape
test.shape

(114393, 132)

In [3]:
# Select numeric columns
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(train.select_dtypes(include=numerics).columns)
train = train[numerical_vars]
train.shape

(114321, 114)

In [4]:
# Select numeric columns
numerics = ['int16','int32','int64','float16','float32','float64']
numerical_vars = list(test.select_dtypes(include=numerics).columns)
test = test[numerical_vars]
test.shape

(114393, 113)

In [5]:
train_label = train['target']
train = train.drop(labels=['target', 'ID'], axis=1)
test = test.drop(labels=['ID'], axis=1)


In [6]:
scaler = StandardScaler()
scaler.fit(train.fillna(0))

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
sel_.fit(scaler.transform(train.fillna(0)), train_label)

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=None)

In [8]:
selected_feat = train.columns[(sel_.get_support())]
print('total features: {}'.format((train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
      np.sum(sel_.estimator_.coef_ == 0)))

total features: 112
selected features: 94
features with coefficients shrank to zero: 18


In [9]:
np.sum(sel_.estimator_.coef_ == 0)

18

In [10]:
removed_feats = train.columns[(sel_.estimator_.coef_ == 0).ravel().tolist()]
removed_feats

Index(['v11', 'v32', 'v41', 'v42', 'v43', 'v53', 'v55', 'v60', 'v64', 'v67',
       'v68', 'v77', 'v83', 'v96', 'v97', 'v103', 'v118', 'v122'],
      dtype='object')

In [11]:
train_selected = sel_.transform(train.fillna(0))
test_selected = sel_.transform(test.fillna(0))
train_selected.shape, test_selected.shape

((114321, 94), (114393, 94))

In [12]:
l1_logit = LogisticRegression(C=1, penalty='l2')
l1_logit.fit(scaler.transform(train.fillna(0)), train_label)

/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/Users/tsawaengsri/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [13]:
np.sum(l1_logit.coef_ == 0)

0